### set the base params

In [2]:
root = '/home/bob/projects/Web3HackerNetwork'
dataset_dir = root + "/data/github/2022-04-07-marine-search"
stats_dir = dataset_dir + "/projects"

### define a method that finds all files with the same name

In [7]:
import os
def find_all(name, path):
    result = []
    for root, dirs, files in os.walk(path):
        if name in files:
            result.append(os.path.join(root, name))
    return result

### find all the commi stat JSON files

In [10]:
commit_files = find_all('commit-stat.log.json', stats_dir)

### load all the files

In [16]:
import json

all_logs = []
for file in commit_files:
    in_stats = json.loads(open(file, 'r').read());
    all_logs.append(in_stats)
print(all_logs[0][0])

{'commit': 'fb528905fb6448e2b502f7de8fba877a2a995f6d', 'Author': 'tchataigner <tom.chataigner@yahoo.fr>', 'Date': 'Sun Aug 15 20', 'fileTypes': {'js': {'textLineCount': 26, 'binByteCount': 0, 'inserts': 17, 'deletes': 9, 'occurrences': 2}}, 'files': '2', 'insertions(+)': '17', 'deletions(-)\n': '9'}


In [43]:
js_commits = []

for log in all_logs:
    #lines.append("\n")
    for commit in log:
        #lines.append(str(commit['commit']))
        #print(commit['fileTypes'])
        for ft in commit['fileTypes']:
            if ft == 'js':
                js_commits.append(commit)

### method to extract statistics from the commit data

In [77]:
def extract_stats(commit):
    if 'files' in commit:
        num_files = commit['files']
    else:
        num_files = commit['file']
    print(num_files)
    typeDict = commit['fileTypes']
    # for x in (y for y in items if y > 10):
    typeArray = [{'fileType': key, 'stats': typeDict[key]} for key in typeDict.keys()]
    textTypeArray = [typeEntry for typeEntry in typeArray if typeEntry['stats']['textLineCount'] > 0]
    binTypeArray = [typeEntry for typeEntry in typeArray if typeEntry['stats']['binByteCount'] > 0]
    textLineCounts = [typeEntry['stats']['textLineCount'] for typeEntry in textTypeArray]
    binByteCounts = [typeEntry['stats']['binByteCount'] for typeEntry in binTypeArray]
    numTextFiles = sum([typeEntry['stats']['occurrences'] for typeEntry in textTypeArray])
    numBinFiles = sum([typeEntry['stats']['occurrences'] for typeEntry in binTypeArray])
    print("total text lines: " + str(sum(textLineCounts)))
    print("total bin bytes: " + str(sum(binByteCounts)))
    print("num text files: " + str(numTextFiles))
    print("num bin files: " + str(numBinFiles))
    # fileType['textLineCount'] for fileType in commit['fileTypes']]

In [78]:
for commit in js_commits[slice(10)]:
    print(commit)
    extract_stats(commit)

{'commit': 'fb528905fb6448e2b502f7de8fba877a2a995f6d', 'Author': 'tchataigner <tom.chataigner@yahoo.fr>', 'Date': 'Sun Aug 15 20', 'fileTypes': {'js': {'textLineCount': 26, 'binByteCount': 0, 'inserts': 17, 'deletes': 9, 'occurrences': 2}}, 'files': '2', 'insertions(+)': '17', 'deletions(-)\n': '9'}
2
total text lines: 26
total bin bytes: 0
num text files: 2
num bin files: 0
{'commit': '31ff2b9db37691a15a89a0c324c89f3a30cf403a', 'Author': 'tchataigner <tom.chataigner@yahoo.fr>', 'Date': 'Sun Aug 15 17', 'fileTypes': {'js': {'textLineCount': 4, 'binByteCount': 0, 'inserts': 2, 'deletes': 2, 'occurrences': 1}}, 'file': '1', 'insertions(+)': '2', 'deletions(-)\n': '2'}
1
total text lines: 4
total bin bytes: 0
num text files: 1
num bin files: 0
{'commit': '6d8a792f9d98776ce2b54bd3664859c48517ce3e', 'Author': 'tchataigner <tom.chataigner@yahoo.fr>', 'Date': 'Sun Aug 15 17', 'fileTypes': {'js': {'textLineCount': 19, 'binByteCount': 0, 'inserts': 13, 'deletes': 6, 'occurrences': 4}}, 'files':